In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Open the website
url = "https://www.thegioididong.com/laptop"
driver.get(url)

# Allow some time for the page to load
time.sleep(3)

# Get page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract all brand URLs (e.g., HP, Asus, Dell)
brand_links = []
brand_list = soup.find_all('a', class_='box-quicklink__item')

for brand in brand_list:
    href = brand.get('data-href')
    if href:
        brand_links.append(href)

# Close the driver after extracting brand URLs
driver.quit()

# Initialize lists to store product details
product_details = []

# Loop through each brand page to extract product data
for brand_url in brand_links:
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    
    # Open the brand page
    driver.get(brand_url)
    time.sleep(3)
    
    # Click on "Xem thêm" if available
    while True:
        try:
            see_more_button = driver.find_element_by_css_selector('div.view-more a')
            see_more_button.click()
            time.sleep(3)  # Allow time for the page to load more products
        except:
            break  # Exit loop if no more "Xem thêm" button is found
    
    # Get page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract all product links from the brand page
    product_list = soup.find_all('a', class_='main-contain')
    product_links = []
    
    for link in product_list:
        href = link.get('href')
        if href:
            product_page_url = 'https://www.thegioididong.com' + href
            review_page_url = product_page_url + "/danh-gia"  # Add "/danh-gia" to check for the review page
            
            # Initialize Chrome driver for review page check
            driver = webdriver.Chrome()
            driver.get(review_page_url)
            time.sleep(3)
            
            # Get the review page content
            soup_review = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Check if review page exists
            error_message = soup_review.find(text="Xin lỗi, chúng tôi không tìm thấy trang mà bạn cần!")
            
            # If the review page exists, add to product_links
            if not error_message:
                product_links.append(review_page_url)
            
            driver.quit()
    
    # Close the driver for the brand page
    driver.quit()
    
    # Now, open each valid product review link to extract details
    for link in product_links:
        driver = webdriver.Chrome()
        driver.get(link)
        time.sleep(3)
        
        # Get page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract product name
        product_name = 'N/A'
        product_name_tag = soup.find('div', class_='content')

        if product_name_tag:
            # Try to find the product name in <h1>, <h2>, <h3>
            h1_tag = product_name_tag.find('h1')
            if h1_tag:
                product_name = h1_tag.text.strip()
            else:
                h2_tag = product_name_tag.find('h2')
                if h2_tag:
                    product_name = h2_tag.text.strip()
                else:
                    h3_tag = product_name_tag.find('h3')
                    if h3_tag:
                        product_name = h3_tag.text.strip()
        
        # Extract current price only
        price_current = 'N/A'
        price_section = soup.find('div', class_='price')
        if price_section:
            price_current_tag = price_section.find('strong', class_='price-current')
            if price_current_tag:
                price_current = price_current_tag.text.strip()

        # Extract reviews if available
        reviews = soup.find_all('li', class_='par')
        
        for review in reviews:
            name = review.find('p', class_='cmt-top-name').text.strip()
            rating = len(review.find_all('i', class_='iconcmt-starbuy'))
            comment = review.find('p', class_='cmt-txt').text.strip()
            time_used_element = review.find('span', class_='cmtd dot-line')
            time_used = time_used_element.text.strip() if time_used_element else 'N/A'

            product_details.append({
                'Product Name': product_name,
                'Current Price': price_current,
                'Reviewer': name,
                'Rating': rating,
                'Comment': comment,
                'Time Used': time_used
            })
        
        driver.quit()

C:\Users\hdang\AppData\Local\Temp\ipykernel_41316\1299149357.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  error_message = soup_review.find(text="Xin lỗi, chúng tôi không tìm thấy trang mà bạn cần!")


In [ ]:
# Convert the collected data to a DataFrame
df = pd.DataFrame(product_details)

# Save the DataFrame to a CSV file
df.to_csv('../data/laptop_data.csv', index=False, encoding='utf-8')

print("Data has been exported to 'laptop_reviews_by_brand.csv'")